Primeiramente vamos realizar os imports:

In [1]:
# import's para implementar o RSA
import numpy as np
import random as rd
from glob import glob
from math import log

# import's para trabalhar com o e-mail
import email
import smtplib
import imaplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

Feito isso, vamos criar uma função para calcular o Algoritmo de Euclides Estendido, isso é, uma função que recebe dois inteiros $a$ e $b$ e retorna $mdc(a, b)$ e valores $x$ e $y$ tais que $a\cdot x + b\cdot y = mdc(a, b)$.

In [2]:
def euclid_extended(a, b):
    inverted = False
    if b > a:
        a, b = b, a
        inverted = True
    
    table = np.array([[a, b], [1, 0], [0, 1]])
    iteration = 0
    while table[0, (iteration + 1) % 2] != 0:
        a, b = table[0, iteration % 2], table[0, (iteration + 1) % 2]
        q = a // b
        table[:, iteration % 2] -= table[:, (iteration + 1) % 2] * q
        iteration += 1
        
    lcd, x, y = table[:, iteration % 2]
    if inverted:
        return lcd, y, x
    else:
        return lcd, x, y

Elaborada tal função, já temos o ferramental para, dados dois primos, gerar os parâmetros para a implementação do RSA, isso é, as duas chaves: pública e privada.

A geração da chave se dará do seguinte modo:
  - dados dois primos $p$ e $q$, calculamos $n = p\cdot q$ e $\phi(n) = (p - 1)\cdot (q - 1)$;
  - feito isso, escolhemos $e$ de modo que $mdc(e, \phi(n)) = 1$ e $2 < e < \phi(n)$;
  - agora, encontramos $d$ de modo que $2 < d < \phi(n)$ e $d\cdot e \equiv 1 \pmod{\phi(n)}$.

Note que os dois últimos passos podem ser realizados simultaneamente via Algoritmo de Euclides Estendido. Dessa forma, a chave pública será dada pelo par $(n, e)$ enquanto a chave privada será dada pelo par $(n, d)$.

In [3]:
def generate_keys(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = rd.randint(3, phi_n)
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1 or d<10**20:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
        
    return (n, e), (n, d)

Tendo as duas chaves, devemos ter funções que vão criptografar e descriptografar uma mensagem. Entretanto, a mensagem é um texto (string), e o RSA trabalha com números, então vamos primeiro criar funções que transformem strings para números e números para strings. A ideia para essas funções será transformar a mensagem para um inteiro em "base" 256 (quantidade de caracteres da tabela ASCII), bem como o caminho inverso. Para isso, estamos usando funções como ```ord``` e ```chr```.

In [4]:
def str2int(message):
    exp = 1
    number = 0
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    
    return message

Tendo essas funções podemos, finalmente, elaborar funções que vão criptografar e descriptografar mensagens:

In [5]:
def encrypt(message, public_key):
    n, e = public_key
    m = str2int(message)
    m = pow(m, e, n)
    encrypted = int2str(m)
    
    return encrypted

def decrypt(encrypted, private_key):
    n, d = private_key
    m = str2int(encrypted)
    m = pow(m, d, n)
    message = int2str(m)
    
    return message

Agora vamos testar as funções. Para isso, criamos uma lista de primos, alguns desses primos foram pegos da [Wikipédia](https://en.wikipedia.org/wiki/Largest_known_prime_number).

In [6]:
primes = [100000000000000003, 100000000000000013, 170141183460469231731687303715884105727, 20988936657440586486151264256610222593863921, 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127, 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151, 10407932194664399081925240327364085538615262247266704805319112350403608059673360298012239441732324184842421613954281007791383566248323464908139906605677320762924129509389220345773183349661583550472959420547689811211693677147548478866962501384438260291732348885311160828538416585028255604666224831890918801847068222203140521026698435488732958028878050869736186900714720710555703168729087]
p, q = rd.choice(primes), rd.choice(primes)

print(p)
print(q)

100000000000000013
100000000000000003


Sabemos que o algoritmo pode não funcionar corretamente caso os primos sejam muito grandes para comportar a mensagem, veja:

In [7]:
text = '''Você sabia? Resolver equações na Roma antiga era bem mais fácil. Afinal, o valor de x era sempre 10.
Estatísticas comprovam: água causa morte. Segundo os cientistas, 100% das mortes ocorrem em seres que bebem água.'''
p, q = primes[-3], primes[-2]
public_key, private_key = generate_keys(p, q)
t = encrypt(text, public_key)
e = decrypt(t, private_key)
m = str2int(text)
print(e)
print()

p, q = primes[-1], primes[-2]
public_key, private_key = generate_keys(p, q)
t = encrypt(text, public_key)
e = decrypt(t, private_key)
print(e)

uCÂæù@Ð¬1

þüþ«þHA¶Å5N 
 ­1þî«ûúSúò­òôÿü÷L­ï  õkþH
É½¦90W	éMaÔ¸X'ðøØ±ßJøuggÈªyBt×ZÅ}ëÞþcà>ïzaS^ßÉåçîïÅéÅbÜ#§

Você sabia? Resolver equações na Roma antiga era bem mais fácil. Afinal, o valor de x era sempre 10.
Estatísticas comprovam: água causa morte. Segundo os cientistas, 100% das mortes ocorrem em seres que bebem água.


Para isso, vamos analisar o maior tamanho de mensagem que é suportada em função de $n$. Para tanto, note que as mensagens são transformadas em um número na base $256$, assim, o maior número pode ter, no máximo, $\lfloor\log_{256} n\rfloor$ dígitos. Pensando nisso, vamos separar as mensagem em trechos que vão possuir entre $\lfloor\frac{\log_{256} n}{2}\rfloor$ e $\lfloor\log_{256} n\rfloor$ caracteres, possibilitando que tenhamos mensagens arbitrariamente grandes.

Dito isso, podemos fazer uma reimplementação das funções para criptografar e descriptografar.

In [8]:
def encrypt(message, public_key):
    n, e = public_key
    m = message
    L = int(log(n, 256))
    l = int(log(n, 256) / 2)
    parts = []
    while len(m) > L:
        r = rd.randint(l, L)
        aux, m = m[:r], m[r:]
        parts.append(aux)
    else:
        parts.append(m)
        
    encrypted = ''
    for part in parts:       
        part = str2int(part)
        part = pow(part, e, n)
        encrypted += chr(256) + int2str(part)
    
    return encrypted

def decrypt(encrypted, private_key):
    encrypted = encrypted.split(chr(256))
    n, d = private_key
    message = ''
    encrypted.remove('')
    for part in encrypted:
        m = str2int(part)
        m = pow(m, d, n)
        message += int2str(m)
    
    return message

Agora, podemos criar uma rotina para utilizarmos o algoritmo implementado acima. A ideia será trocar e-mails criptografados. Para tanto, criamos uma função que recebe um usuário e senha, além de um endereço de e-mail de destinatário, assunto, corpo do e-mail e a chave pública. Para simplificar na localização do e-mail que vamos descriptografar para ler, essa função também recebe como parâmetro uma tag para marcar esse e-mail.

In [9]:
def send_mail(username, password, mail_address, subject, body, public_key, tag = '[Encrypted] '):
    body = encrypt(body, public_key)
    subject = tag + subject
    msg = MIMEMultipart()
    msg['From'] = username
    msg['To'] = mail_address
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(username, password)
    text = msg.as_string()
    server.sendmail(username, mail_address, text)
    server.quit()

Tendo enviado o e-mail, criamos uma função que recebe o usuário e a senha de um e-mail, além da chave privada e lê os e-mails criptografados (marcados com a tag no assunto).

In [10]:
def read_email(username, password, private_key, tag = '[Encrypted]', qtd = 1):
    printed = 0
    crip = False
    server = 'imap.gmail.com'
    mail = imaplib.IMAP4_SSL(server)
    mail.login(username, password)
    mail.select('inbox')
    data = mail.search(None, 'ALL')
    mail_ids = data[1]
    id_list = mail_ids[0].split()   
    first_email_id = int(id_list[0])
    latest_email_id = int(id_list[-1])
    for i in range(latest_email_id, first_email_id, -1):
        if printed == qtd:
            break

        data = mail.fetch(str(i), '(RFC822)')
        for response_part in data:
            arr = response_part[0]
            if isinstance(arr, tuple):
                msg = email.message_from_string(str(arr[1], 'utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                if tag not in email_subject:
                    break
                else:
                    printed += 1
                    crip = True

                print('From: ' + email_from)
                print('Subject: ' + email_subject)
                print()
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get('Content-Disposition'))
                    try:
                        body = part.get_payload(decode = True).decode()
                    except:
                        pass

                    if content_type == 'text/plain':
                        body = decrypt(body, private_key)
                        print(body)
        
        if printed != qtd and crip:
            print()
            crip = False

Agora, temos um código para gerar e salvar as chaves. A ideia de salvar as chaves é para possibilitar que carreguemos as mesmas, podendo ler os e-mails mesmo após fechar esse notebook, caso contrário perderíamos as duas chaves.

In [11]:
if 'public.csv' not in glob('*.csv'):
    p, q = rd.choice(primes), rd.choice(primes)
    while 1/5 < p/q < 5:
        p, q = rd.choice(primes), rd.choice(primes)
    public_key, private_key = generate_keys(p, q)
    with open('public.csv', 'w') as file:
        file.write(str(public_key[0]) + '\n')
        file.write(str(public_key[1]) + '\n')
        
    with open('private.csv', 'w') as file:
        file.write(str(private_key[0]) + '\n')
        file.write(str(private_key[1]) + '\n')   
else:
    file = open('public.csv')
    public_key = file.readlines()
    file.close()
    public_key = tuple([int(i) for i in public_key])
    
    file = open('private.csv')
    private_key = file.readlines()
    file.close()
    private_key = tuple([int(i) for i in private_key])

Agora, vamos pegar as credenciais do usuário:

In [12]:
username = input('Username: ')
password = getpass(prompt = 'Password: ')

Username: igorpmichels@gmail.com
Password: ········


E, finalmente, vamos enviar um e-mail utilizando o RSA:

In [13]:
body = '''Não tinha medo o tal João de Santo Cristo
Era o que todos diziam quando ele se perdeu
Deixou pra trás todo o marasmo da fazenda
Só pra sentir no seu sangue o ódio que Jesus lhe deu
Quando criança só pensava em ser bandido
Ainda mais quando com um tiro de soldado o pai morreu
Era o terror da sertania onde morava
E na escola até o professor com ele aprendeu
Ia pra igreja só pra roubar o dinheiro
Que as velhinhas colocavam na caixinha do altar
Sentia mesmo que era mesmo diferente
Sentia que aquilo ali não era o seu lugar
Ele queria sair para ver o mar
E as coisas que ele via na televisão
Juntou dinheiro para poder viajar
De escolha própria, escolheu a solidão
Comia todas as menininhas da cidade
De tanto brincar de médico, aos doze era professor
Aos quinze, foi mandado pro o reformatório
Onde aumentou seu ódio diante de tanto terror
Não entendia como a vida funcionava
Discriminação por causa da sua classe e sua cor
Ficou cansado de tentar achar resposta
E comprou uma passagem, foi direto a Salvador
E lá chegando foi tomar um cafezinho
E encontrou um boiadeiro com quem foi falar
E o boiadeiro tinha uma passagem e ia perder a viagem
Mas João foi lhe salvar
Dizia ele "estou indo pra Brasília
Neste país lugar melhor não há
'Tô precisando visitar a minha filha
Eu fico aqui e você vai no meu lugar"
E João aceitou sua proposta
E num ônibus entrou no Planalto Central
Ele ficou bestificado com a cidade
Saindo da rodoviária, viu as luzes de Natal
Meu Deus, mas que cidade linda
No Ano Novo eu começo a trabalhar
Cortar madeira, aprendiz de carpinteiro
Ganhava cem mil por mês em Taguatinga
Na sexta-feira ia pra zona da cidade
Gastar todo o seu dinheiro de rapaz trabalhador
E conhecia muita gente interessante
Até um neto bastardo do seu bisavô
Um peruano que vivia na Bolívia
E muitas coisas trazia de lá
Seu nome era Pablo e ele dizia
Que um negócio ele ia começar
E o Santo Cristo até a morte trabalhava
Mas o dinheiro não dava pra ele se alimentar
E ouvia às sete horas o noticiário
Que sempre dizia que o seu ministro ia ajudar
Mas ele não queria mais conversa
E decidiu que, como Pablo, ele ia se virar
Elaborou mais uma vez seu plano santo
E sem ser crucificado, a plantação foi começar
Logo logo os maluco da cidade souberam da novidade
"Tem bagulho bom ai!"
E João de Santo Cristo ficou rico
E acabou com todos os traficantes dali
Fez amigos, frequentava a Asa Norte
E ia pra festa de rock, pra se libertar
Mas de repente Sob uma má influência dos boyzinho da cidade
Começou a roubar
Já no primeiro roubo ele dançou
E pro inferno ele foi pela primeira vez
Violência e estupro do seu corpo
Vocês vão ver, eu vou pegar vocês
Agora o Santo Cristo era bandido
Destemido e temido no Distrito Federal
Não tinha nenhum medo de polícia
Capitão ou traficante, playboy ou general
Foi quando conheceu uma menina
E de todos os seus pecados ele se arrependeu
Maria Lúcia era uma menina linda
E o coração dele pra ela o Santo Cristo prometeu
Ele dizia que queria se casar
E carpinteiro ele voltou a ser
Maria Lúcia pra sempre vou te amar
E um filho com você eu quero ter
O tempo passa e um dia vem na porta
Um senhor de alta classe com dinheiro na mão
E ele faz uma proposta indecorosa
E diz que espera uma resposta, uma resposta do João
Não boto bomba em banca de jornal
Nem em colégio de criança isso eu não faço não
E não protejo general de dez estrelas
Que fica atrás da mesa com o cu na mão
E é melhor senhor sair da minha casa
Nunca brinque com um Peixes de ascendente Escorpião"
Mas antes de sair, com ódio no olhar, o velho disse
"Você perdeu sua vida, meu irmão"
"Você perdeu a sua vida meu irmão
Você perdeu a sua vida meu irmão
Essas palavras vão entrar no coração
Eu vou sofrer as consequências como um cão"
Não é que o Santo Cristo estava certo
Seu futuro era incerto e ele não foi trabalhar
Se embebedou e no meio da bebedeira
Descobriu que tinha outro trabalhando em seu lugar
Falou com Pablo que queria um parceiro
E também tinha dinheiro e queria se armar
Pablo trazia o contrabando da Bolívia
E Santo Cristo revendia em Planaltina
Mas acontece que um tal de Jeremias
Traficante de renome, apareceu por lá
Ficou sabendo dos planos de Santo Cristo
E decidiu que, com João ele ia acabar
Mas Pablo trouxe uma Winchester-22
E Santo Cristo já sabia atirar
E decidiu usar a arma só depois
Que Jeremias começasse a brigar
Jeremias, maconheiro sem-vergonha
Organizou a Rockonha e fez todo mundo dançar
Desvirginava mocinhas inocentes
Se dizia que era crente mas não sabia rezar
E Santo Cristo há muito não ia pra casa
E a saudade começou a apertar
Eu vou me embora, eu vou ver Maria Lúcia
Já 'tá em tempo de a gente se casar
Chegando em casa então ele chorou
E pro inferno ele foi pela segunda vez
Com Maria Lúcia Jeremias se casou
E um filho nela ele fez
Santo Cristo era só ódio por dentro
E então o Jeremias pra um duelo ele chamou
Amanhã às duas horas na Ceilândia
Em frente ao Lote 14, é pra lá que eu vou
E você pode escolher as suas armas
Que eu acabo mesmo com você, seu porco traidor
E mato também Maria Lúcia
Aquela menina bosal pra quem jurei o meu amor
E o Santo Cristo não sabia o que fazer
Quando viu o repórter da televisão
Que deu notícia do duelo na TV
Dizendo a hora e o local e a razão
No sábado então, às duas horas
Todo o povo sem demora foi lá só para assistir
Um homem que atirava pelas costas
E acertou o Santo Cristo, começou a sorrir
Sentindo o sangue na garganta
João olhou pras bandeirinhas e pro povo a aplaudir
E olhou pro sorveteiro e pras câmeras e
A gente da TV que filmava tudo ali
E se lembrou de quando era uma criança
E de tudo o que vivera até ali
E decidiu entrar de vez naquela dança
Se a via-crucis virou circo, estou aqui
E nisso o sol cegou seus olhos
E então Maria Lúcia ele reconheceu
Ela trazia a Winchester-22
A arma que seu primo Pablo lhe deu
Jeremias, eu sou homem. coisa que você não é
E não atiro pelas costas não
Olha pra cá filha da puta, sem-vergonha
Dá uma olhada no meu sangue e vem sentir o teu perdão
E Santo Cristo com a Winchester-22
Deu cinco tiros no bandido traidor
Maria Lúcia se arrependeu depois
E morreu junto com João, seu protetor
E o povo declarava que João de Santo Cristo
Era santo porque sabia morrer
E a alta burguesia da cidade
Não acreditou na história que eles viram na TV
E João não conseguiu o que queria
Quando veio pra Brasília, com o diabo ter
Ele queria era falar pro presidente
Pra ajudar toda essa gente que só faz
Sofrer'''

mail_address = username # 'luca.escopelli@gmail.com' # destinatário (igual o username para ler o e-mail depois)
subject = 'Faroeste Caboclo'

send_mail(username, password, mail_address, subject, body, public_key)

Agora, vamos ler o e-mail que acabamos de enviar e que foi criptografado:

In [14]:
read_email(username, password, private_key)

From: igorpmichels@gmail.com
Subject: [Encrypted] Faroeste Caboclo

Não tinha medo o tal João de Santo Cristo
Era o que todos diziam quando ele se perdeu
Deixou pra trás todo o marasmo da fazenda
Só pra sentir no seu sangue o ódio que Jesus lhe deu
Quando criança só pensava em ser bandido
Ainda mais quando com um tiro de soldado o pai morreu
Era o terror da sertania onde morava
E na escola até o professor com ele aprendeu
Ia pra igreja só pra roubar o dinheiro
Que as velhinhas colocavam na caixinha do altar
Sentia mesmo que era mesmo diferente
Sentia que aquilo ali não era o seu lugar
Ele queria sair para ver o mar
E as coisas que ele via na televisão
Juntou dinheiro para poder viajar
De escolha própria, escolheu a solidão
Comia todas as menininhas da cidade
De tanto brincar de médico, aos doze era professor
Aos quinze, foi mandado pro o reformatório
Onde aumentou seu ódio diante de tanto terror
Não entendia como a vida funcionava
Discriminação por causa da sua classe e sua cor
Ficou c